TEXT

In [1]:
# IMPORTS
import random
import numpy as np
import pandas as pd

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from ProblemScripts import load_problem_KP

In [2]:
# Set random seed to ensure reproducibility
random.seed(37)

In [3]:
n_items, capacity, optimal, values, weights, items_dict = load_problem_KP('f1_l-d_kp_10_269')
# load_problem_KP('knapPI_1_100_1000_1')

number of items: 10
max weight: 269
values: [55 10 47  5  4 50  8 61 85 87]
weights: [95  4 60 32 23 72 80 62 65 46]
optimal solution: 295


In [4]:
def knapsack_fitness(solution, values, weights, capacity):
    total_weight = np.dot(solution, weights)
    total_value = np.dot(solution, values)
    if total_weight > capacity:
        return 0  # Invalid solution
    return total_value

In [5]:
def generate_random_solution(length):
    return np.random.randint(2, size=length)

In [6]:
test_sol = generate_random_solution(n_items)
test_sol_fitness = knapsack_fitness(test_sol, values, weights, capacity)
print(test_sol)
print(test_sol_fitness)

[1 1 0 0 0 0 0 0 1 0]
150


In [7]:
def evalIndividual(individual):
    weight = 0
    value = 0
    for i in range(n_items):
        value += items_dict[i][0] * individual[i]
        weight += items_dict[i][1] * individual[i]
    if weight > capacity:
        return (0,)
    return (value,)

def BasicEA(pop_size, n_generations):
    # Create fitness class
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    # weights is positive to signify maximisation of fitness, use negative to minimise
    # weights must be a tuple to allow to single and multi objective problems
    # to be treated in the same way

    # create individual class
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox() # Initialise DEAP toolbox
    toolbox.register("attr_int", random.randint, 0, 1) # generate atributes of 0 or 1
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=n_items)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    # toolbox.register("select", tools.selBest)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evalIndividual)

    gen = 0
    # initialise population
    population = toolbox.population(pop_size)
    # print(population)

    ngen, cxpb, mutpb = n_generations, 0.5, 0.2  # Number of generations, crossover probability, mutation probability

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(f[0] for f in x) / len(x))
    stats.register("max", lambda x: max(f[0] for f in x))

    algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=stats, halloffame=hof, verbose=True)

    best_ind = hof[0]
    print(f"Best individual: {best_ind}, Value: {evalIndividual(best_ind)[0]}")


In [19]:
print(f'Optimal Solution: {optimal}')
BasicEA(100, 10)

Optimal Solution: 295
gen	nevals	avg  	max
0  	100   	85.77	232
1  	61    	131.87	241
2  	49    	175.09	284
3  	56    	176.85	285
4  	67    	215.74	285
5  	70    	227.28	295
6  	66    	246.99	295
7  	68    	245.96	295
8  	60    	260.89	295
9  	70    	256.57	295
10 	54    	271.48	295
Best individual: [0, 1, 1, 1, 0, 0, 0, 1, 1, 1], Value: 295


In [22]:
def BasicEA_TT(pop_size, n_generations):
    # Create fitness class
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    # weights is positive to signify maximisation of fitness, use negative to minimise
    # weights must be a tuple to allow to single and multi objective problems
    # to be treated in the same way

    # create individual class
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox() # Initialise DEAP toolbox
    toolbox.register("attr_int", random.randint, 0, 1) # generate atributes of 0 or 1
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=n_items)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    # toolbox.register("select", tools.selBest)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evalIndividual)

    gen = 0
    # initialise population
    population = toolbox.population(pop_size)
    # print(population)

    ngen, cxpb, mutpb = n_generations, 0.5, 0.2  # Number of generations, crossover probability, mutation probability

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(f[0] for f in x) / len(x))
    stats.register("max", lambda x: max(f[0] for f in x))

    trajectory = []  # Store the population at each generation (nodes)
    transitions = []  # Store the transitions between generations (edges)

    for gen in range(ngen):
        population = algorithms.varAnd(population, toolbox, cxpb, mutpb) # apply crossover and mutation
        fits = [toolbox.evaluate(ind)[0] for ind in population] # evaluate fitness of population

        # Log the trajectory of solutions
        trajectory.append([tuple(ind) for ind in population])

        # update fitness
        for ind, fit in zip(population, fits):
            ind.fitness.values = (fit,)
        
        # Update Hall of Fame
        hof.update(population)

        # Select the next generation population
        next_population = toolbox.select(population, len(population))

        # Record transitions between solutions
        for i, (ind, next_ind) in enumerate(zip(population, next_population)):
            transitions.append((tuple(ind), tuple(next_ind), toolbox.evaluate(next_ind)[0] - toolbox.evaluate(ind)[0]))

        population[:] = next_population # update population
    
    best_ind = hof[0]
    print(f"Best individual: {best_ind}, Value: {evalIndividual(best_ind)[0]}")

    return trajectory, transitions


In [25]:
print(f'Optimal Solution: {optimal}')
BasicEA_TT(100, 10)

Optimal Solution: 295
Best individual: [0, 1, 1, 1, 0, 0, 0, 1, 1, 1], Value: 295


([[(1, 1, 0, 0, 0, 0, 1, 0, 0, 1),
   (0, 0, 1, 0, 0, 1, 1, 1, 0, 0),
   (1, 0, 1, 1, 0, 0, 0, 0, 0, 1),
   (0, 0, 0, 0, 0, 1, 0, 0, 1, 0),
   (1, 1, 1, 1, 1, 0, 1, 1, 1, 1),
   (0, 0, 1, 0, 1, 1, 0, 0, 1, 1),
   (0, 0, 0, 0, 0, 0, 0, 0, 1, 1),
   (1, 1, 1, 1, 1, 0, 0, 0, 0, 1),
   (0, 1, 1, 1, 1, 0, 1, 1, 0, 0),
   (1, 0, 1, 0, 0, 1, 1, 0, 0, 0),
   (1, 0, 1, 0, 1, 1, 1, 1, 1, 0),
   (1, 1, 0, 0, 0, 1, 1, 1, 1, 0),
   (0, 1, 1, 1, 0, 1, 1, 1, 0, 0),
   (0, 1, 1, 1, 0, 1, 1, 1, 0, 0),
   (0, 1, 0, 0, 0, 1, 0, 0, 1, 0),
   (0, 0, 0, 0, 0, 1, 0, 1, 1, 1),
   (1, 1, 0, 1, 1, 1, 1, 1, 1, 1),
   (1, 1, 0, 1, 1, 1, 0, 1, 0, 0),
   (1, 0, 1, 0, 1, 0, 1, 1, 0, 0),
   (1, 1, 0, 0, 1, 1, 0, 0, 1, 1),
   (1, 1, 1, 1, 1, 0, 0, 0, 1, 1),
   (1, 1, 1, 0, 0, 1, 1, 1, 0, 1),
   (1, 0, 0, 1, 1, 0, 0, 1, 1, 0),
   (0, 1, 1, 1, 0, 0, 0, 1, 0, 1),
   (1, 1, 0, 0, 1, 1, 0, 0, 1, 0),
   (1, 1, 0, 0, 1, 0, 0, 1, 1, 0),
   (1, 0, 0, 1, 0, 0, 0, 0, 0, 0),
   (0, 1, 1, 0, 0, 0, 0, 1, 1, 0),
   (0, 1, 1, 0, 0, 0